In [525]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer


In [526]:
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

#submission = pd.read_csv("house-prices-advanced-regression-techniques/sample_submission.csv")

In [527]:
print("Full train dataset shape is {}".format(train.shape))

Full train dataset shape is (1460, 81)


In [528]:
train = train.drop(columns="Id")

In [1]:
train.head(10)

NameError: name 'train' is not defined

In [530]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [531]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [532]:
train.isna().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [533]:
# Identifiziere Spalten mit mehr als 200 fehlenden Werten
cols_with_many_nas = train.columns[train.isnull().sum() > 200]

print("Spalten mit mehr als 200 NA-Werten:")
print(cols_with_many_nas)

Spalten mit mehr als 200 NA-Werten:
Index(['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')


In [534]:
# Drop columns with more than 200 missing values
train = train.dropna(axis=1, thresh=len(train) - 200)

In [535]:
train.isna().sum()

MSSubClass       0
MSZoning         0
LotArea          0
Street           0
LotShape         0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 74, dtype: int64

In [536]:
# drop all rows with NA
#train.dropna(inplace=True)
#print("Aktuelle Größe des DataFrames nach Entfernen der Zeilen mit fehlenden Werten:", train.shape)

In [537]:
# Split the data into predictors and target
X = train.drop(columns=["SalePrice"])
y = train["SalePrice"]

In [538]:
print(X)

      MSSubClass MSZoning  LotArea Street LotShape LandContour Utilities  \
0             60       RL     8450   Pave      Reg         Lvl    AllPub   
1             20       RL     9600   Pave      Reg         Lvl    AllPub   
2             60       RL    11250   Pave      IR1         Lvl    AllPub   
3             70       RL     9550   Pave      IR1         Lvl    AllPub   
4             60       RL    14260   Pave      IR1         Lvl    AllPub   
...          ...      ...      ...    ...      ...         ...       ...   
1455          60       RL     7917   Pave      Reg         Lvl    AllPub   
1456          20       RL    13175   Pave      Reg         Lvl    AllPub   
1457          70       RL     9042   Pave      Reg         Lvl    AllPub   
1458          20       RL     9717   Pave      Reg         Lvl    AllPub   
1459          20       RL     9937   Pave      Reg         Lvl    AllPub   

     LotConfig LandSlope Neighborhood  ... OpenPorchSF EnclosedPorch  \
0       Inside 

In [539]:
# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

In [540]:
# Create transformers and categorical columns
numerical_transformer = StandardScaler()
categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [541]:
# columns transformer to apply transformers to the appropriate columns
# Hier füge sowohl den SimpleImputer für kategoriale Spalten als auch den IterativeImputer für numerische Spalten hinzu
numerical_pipeline = Pipeline([
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ])


In [542]:
# Define the model
model = RandomForestRegressor(n_estimators=2, random_state=42)

In [543]:
# Create and evaluate the pipeline
pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                           ("model", model)
                           ])

In [544]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [545]:
# Preprocessing of training data and train model
pipeline.fit(X_train, y_train)

# Preprocess the test data and make predictions
y_pred = pipeline.predict(X_test)

In [546]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 817800399.3042753


In [547]:
r2_score_train = pipeline.score(X_train, y_train)
print(f"R^2 Score for train_data: {r2_score_train}")

R^2 Score for train_data: 0.9792243538186987


In [548]:
r2_score_test = pipeline.score(X_test, y_test)
print(f"R^2 Score for test_data: {r2_score_test}")

R^2 Score for test_data: 0.8933812751616752


In [549]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [550]:
# Entfernen Sie Spalten aus `test`, die mehr als 200 fehlende Werte haben
test = test.dropna(axis=1, thresh=len(test) - 200)

# Speichern Sie die 'Id'-Werte für spätere Verwendung
ids = test.Id
test = test.drop("Id", axis=1)

In [551]:
# Machen Sie die Vorhersagen
predict_test = pipeline.predict(test)

# Erstellen Sie einen DataFrame für die Ausgabe
output = pd.DataFrame({'Id': ids,
                       'SalePrice': predict_test.squeeze()})

# Zeigen Sie die ersten Zeilen des Ausgabe-DataFrames an
print(output.head())

     Id  SalePrice
0  1461  128208.83
1  1462  153430.50
2  1463  178591.35
3  1464  187842.19
4  1465  205381.30


In [552]:
output.to_csv("submission.csv", index=False)

Damit habe ich Position 2413 von 4234 erreicht